In [1]:
import re
import string
import numpy as np

In [2]:
#准备去除标点符号和停用词
punctuation_string=string.punctuation
stopwords=['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

In [99]:
#读取文件 labels用来放邮件分类标签 data用来放每句话及里面的单词
def get_data_labels():
    data = []
    labels = []
    y=[]

    with open('SMSSpamCollection.txt','r',encoding = 'utf-8') as f:
        for line in f.readlines():
            line=line.lower()#全部转换成小写字母
            line=re.sub('[{}]'.format(punctuation_string),"",line)#把英文标点符号去掉
            line = line.strip("\n").split()#去除首尾换行符，并按照空格划分
            labels.append(line[0])
            data.append(line[1:])
            y.append(line[0])
    return data,labels,y

In [4]:
#过滤停用词
def filterate(data):
    new_data=[]
    for line in data:
        line=[w for w in line if w not in stopwords]
        new_data.append(line)
    return new_data

In [5]:
#把分类数字化 得到类别的一个向量
def get_labels_vec(labels):
    class_dict={"spam":1,"ham":0}
    labels_vec=[]
    for cls in labels:
        labels_vec.append(class_dict[cls])
    return labels_vec

In [6]:
#创建一个词汇字典
def get_vocablist(new_data):
    vocablist=[]
    for line in new_data:
        for word in line:
            if word not in vocablist:
                vocablist.append(word) 
    return vocablist

In [34]:
#把文本向量化
def get_document_vec(new_data,vocablist):
    document_vec=[]
    for line in new_data:
        words_vec=[0]*len(vocablist)
        for word in line:
            if word in vocablist:
                words_vec[vocablist.index(word)]=1#单词在字典里面的下标索引
        document_vec.append(words_vec)
    return document_vec

In [35]:
#计算各个类别下每个词的概率
def compute_p(new_data,vocablist,labels_vec,document_vec):
    num_lines=len(new_data)#一共多少条邮件
    num_words=len(vocablist)#词汇表的长度
    p_spam=sum(labels_vec)/num_lines #垃圾邮件的概率p(c) 垃圾邮件加起来 除以 邮件总数

    p0_Num=np.ones(num_words)#p0 =p(x|c0) 某个类别下词汇出现概率 长度等于词汇表长度，出现，则相应位置+1
    p1_Num=np.ones(num_words)#p1=p(x|c1)
    p0_Denom=2.0
    p1_Denom=2.0
    
    
    for i in range(num_lines):#遍历每条邮件
        if labels_vec[i]==1:#如果对应得分类是垃圾邮件，那么
            p1_Num+=document_vec[i]#出现的词汇在对应位置＋1
            p1_Denom+=sum(document_vec[i])#分母是这条邮件出现的总词数
        else:
            p0_Num+=document_vec[i]
            p0_Denom+=sum(document_vec[i])

    p1Vec=np.log(p1_Num/p1_Denom)
    p0Vec=np.log(p0_Num/p0_Denom)#分式是一个长度等于词汇表的向量，其中的每个值是对应词汇在这个类别条件下发生的概率

    return p_spam,p1Vec,p0Vec

In [68]:
#分类器 
def classify_final(get_document_vec,sample_handled):
    sample_vec=get_document_vec(sample_handled,vocablist)
    p1=sum(np.dot(sample_vec,p1Vec.T))+np.log(p_spam)#sample_vec 每个词
    p0=sum(np.dot(sample_vec,p0Vec.T))+np.log(1-p_spam)
    if p1>p0:
        return "spam"
    else:
        return "ham"

In [71]:
#对验证样本的处理
def handle(sample):
    sample_handled=[]
    for line in sample:
        line=line.lower()#全部转换成小写字母
        line=re.sub('[{}]'.format(punctuation_string),"",line)#把英文标点符号去掉
        line = line.strip("\n").split()#去除首尾换行符，并按照空格划分
        sample_handled.append(line)
        
    return sample_handled

In [100]:
data,labels,y=get_data_labels()
new_data=filterate(data)
labels_vec=get_labels_vec(labels)
vocablist=get_vocablist(new_data)
document_vec=get_document_vec(new_data,vocablist)
p_spam,p1Vec,p0Vec=compute_p(new_data,vocablist,labels_vec,document_vec)
p_spam,p1Vec,p0Vec

(0.1340150699677072,
 array([-5.93282526, -9.39856116, -9.39856116, ..., -9.39856116,
        -9.39856116, -9.39856116]),
 array([-5.10594547, -9.85953567, -7.91362552, ..., -9.85953567,
        -9.85953567, -9.85953567]))

In [88]:
sample=["Customer service annoncement. You have a New Years delivery waiting for you. Please call 07046744435 now to arrange delivery"]
sample_handled=handle(sample)
result=classify_final(get_document_vec,sample_handled)
print(result)

[['customer', 'service', 'annoncement', 'you', 'have', 'a', 'new', 'years', 'delivery', 'waiting', 'for', 'you', 'please', 'call', '07046744435', 'now', 'to', 'arrange', 'delivery']]
spam


In [92]:

results=[]
for line in data:
    results.append(classify_final(get_document_vec,[line]))
    

In [105]:
n=len(y)

In [108]:
err=0
for i in range(n):
    if(results[i]!=y[i]):
         err+=1
#错误率
err_rate=err/n
err_rate

0.01740222461428059